# Read BVH file ✓

In [ ]:
from Motion.BVH import load

animation, joints_names, frametime = load("./test.bvh")

# Write BVH file ✓

In [ ]:
from Motion.BVH import save

save(filename="test.bvh", anim=animation, names=joints_names, frametime=frametime)

# Edit BVH file (center animation) ✓

In [ ]:
from Motion.BVH import load, save

animation, joints_names, frametime = load("./test.bvh")
animation.positions[:, :, [0, 2]] -= animation.positions[0, 0, [0, 2]]
save(filename="centered.bvh", anim=animation, names=joints_names, frametime=frametime)

# Convert BVH data to XYZ positions ✓

In [ ]:
from Motion.Animation import positions_global
from Motion.BVH import load

animation, joints_name, frametime = load("./test.bvh")
anim_xyz = positions_global(animation)
print(anim_xyz)

# Get kinematic chain from BVH ✓

In [ ]:
from Motion.BVH import load
from Motion.AnimationStructure import get_kinematic_chain

animation, joints_name, frametime = load("./test.bvh")
kinematic_chain = get_kinematic_chain(animation.parents)
print(kinematic_chain)

# Visualize motion ✓

In [ ]:
from Motion.Animation import positions_global
from Motion.AnimationStructure import get_kinematic_chain
from Motion.BVH import load
from Motion.plot_script import plot_3d_motion


animation, joints_name, frametime = load("./test.bvh")
skeleton = get_kinematic_chain(animation.parents)
anim_xyz = positions_global(animation)

plot_3d_motion("test_vis.mp4", skeleton, anim_xyz[:600], title="Test viz", fps=100)

# Convert angles to various representations ✓

In [ ]:
import torch
import Motion.transforms as tr
from Motion.Animation import Animation
from Motion.BVH import load, save
from Motion.Quaternions import Quaternions


animation, joints_names, frametime = load("./test.bvh")
n_frames, n_joints = animation.shape
print(f"Animation loaded\nn frames: {n_frames}\tn_joints: {n_joints}")

# animation.rotations is by default represented as Quaternions
rotations = torch.tensor(animation.rotations.qs)
print("Quaternions shape:", rotations.shape)

# # Convert quaternions to 6D representation...
rotations = tr.quat2repr6d(rotations)
print("Repr6d shape:", rotations.shape)

# ... and back to quaternions
rotations = tr.repr6d2quat(rotations)

# and now to Euler angles
rotations = tr.quat2euler(rotations, order="xyz", degrees=False)
print("Euler shape:", rotations.shape)

# Now export back to BVH... and it should be the exact same :)
new_anim = Animation(
    Quaternions.from_euler(rotations.numpy(), order="xyz", world=False),
    animation.positions,
    animation.orients,
    animation.offsets,
    animation.parents,
)
save("test_angular_conversions.bvh", new_anim, joints_names, frametime)

# XYZ to rotations

In [ ]:
from Motion.Animation import Animation, positions_global
from Motion.BVH import load, save
from Motion.Quaternions import Quaternions


animation, joints_name, frametime = load("./test.bvh")
anim_xyz = positions_global(animation)

In [ ]:
# First try without optimization
import numpy as np
from scipy.spatial.transform import Rotation as R


def compute_joint_rotations(anim_xyz, offsets, parents):
    n_frames, n_joints, _ = anim_xyz.shape
    rotations = np.zeros((n_frames, n_joints, 3))  # Store Euler angles

    # Loop over frames
    for f in range(n_frames):
        for j in range(n_joints):
            # Root joint (no parent)
            if parents[j] == -1:
                continue

            # Compute the vector from parent to joint in current frame
            parent_pos = anim_xyz[f, parents[j]]
            joint_pos = anim_xyz[f, j]
            current_vector = joint_pos - parent_pos

            # Normalize the vector
            current_vector /= np.linalg.norm(current_vector)

            # Get the rest pose offset (direction of the joint in T-pose)
            rest_vector = offsets[j]
            rest_vector /= np.linalg.norm(rest_vector)

            # Calculate the rotation matrix from rest pose to current pose
            rot_matrix = compute_rotation_matrix(rest_vector, current_vector)

            # Convert the rotation matrix to Euler angles
            # Assuming XYZ convention for Euler angles
            euler_angles = R.from_matrix(rot_matrix).as_euler("xyz", degrees=True)

            # Store the Euler angles
            rotations[f, j] = euler_angles

    return rotations


def compute_rotation_matrix(v1, v2):
    """
    Compute the rotation matrix that aligns vector v1 to v2.
    """
    v1 = np.array(v1)
    v2 = np.array(v2)

    # Cross product to find the axis of rotation
    axis = np.cross(v1, v2)
    axis_norm = np.linalg.norm(axis)

    # Check if the vectors are already aligned
    if axis_norm == 0:
        return np.eye(3)

    axis /= axis_norm

    # Angle between vectors
    angle = np.arccos(np.dot(v1, v2))

    # Use Rodrigues' rotation formula to compute the rotation matrix
    K = np.array(
        [[0, -axis[2], axis[1]], [axis[2], 0, -axis[0]], [-axis[1], axis[0], 0]]
    )
    rot_matrix = np.eye(3) + np.sin(angle) * K + (1 - np.cos(angle)) * np.dot(K, K)

    return rot_matrix


rotations = compute_joint_rotations(anim_xyz, animation.offsets, animation.parents)
new_anim = Animation(
    Quaternions.from_euler(np.deg2rad(rotations), order="xyz", world=False),
    animation.positions,
    animation.orients,
    animation.offsets,
    animation.parents,
)
save("test_rot2xyz.bvh", new_anim, joints_name, frametime)
# not working for now...

In [ ]:
# 2nd try with scipy optimization
from scipy.optimize import minimize


def objective_function(euler_angles, target_position, parent_position, offset):
    """
    Objective function to minimize the difference between target_position (anim_xyz)
    and the position reconstructed from the Euler angles and parent joint position.
    """
    # Convert Euler angles to rotation matrix
    rotation_matrix = R.from_euler("xyz", euler_angles, degrees=True).as_matrix()

    # Compute the predicted position of the joint
    predicted_position = parent_position + np.dot(rotation_matrix, offset)

    # Compute the error (distance between predicted and actual position)
    error = np.linalg.norm(predicted_position - target_position)

    return error


def optimize_rotations(anim_xyz, offsets, parents):
    n_frames, n_joints, _ = anim_xyz.shape
    optimized_rotations = np.zeros((n_frames, n_joints, 3))  # Euler angles

    # Loop over frames and optimize joint rotations
    for f in range(n_frames):
        for j in range(n_joints):
            if parents[j] == -1:
                continue

            # Get the parent and target positions
            parent_pos = anim_xyz[f, parents[j]]
            target_pos = anim_xyz[f, j]

            # Initial guess for the Euler angles
            initial_guess = np.zeros(3)  # Start with no rotation

            # Optimize Euler angles to minimize the error
            result = minimize(
                objective_function,
                initial_guess,
                args=(target_pos, parent_pos, offsets[j]),
                method="BFGS",
            )

            # Store the optimized Euler angles
            optimized_rotations[f, j] = result.x

    return optimized_rotations


animation, joints_name, frametime = load("./test.bvh")
anim_xyz = positions_global(animation)
rots = optimize_rotations(anim_xyz, animation.offsets, animation.parents)